# Example 7
## Impulse response reconstruction
### Generate a full audio spectrum plane from sampled unity plane wave

In [1]:
import numpy as np
import sys
sys.path.insert(0, '../')
from sound_field_analysis import gen, process, plot

from plotly.offline import download_plotlyjs, init_notebook_mode
init_notebook_mode(connected=True)

In [2]:
pi = np.pi
Nsft = 5     # Spatial fourier transform order
Nrf = Nsft   # Radial filter order
Npdc = Nsft  # Decomposition order
OmegaL = np.array([[0, pi / 2],  # Looking directions for plane wave decomposition
                   [pi / 2, pi / 2]])
limit = 150  # Amplification limit of radial filters
r = 0.2      # Array radius
ac = 2       # Rigid Sphere Array
FS = 24000   # Sampling Frequency
NFFT = 1024  # FFT-Bins
AZ = 0       # Azimuth angle
EL = pi / 2  # Elevation angle

## Generate Lebedev grid of order 110

In [3]:
quadrature_grid, _ = gen.lebedev(110)

Lebedev Grid


## Retrieve FFT of sampled plane wave

In [4]:
fftData, kr = gen.sampledWave(r=r, gridData=quadrature_grid, ac=ac, FS=FS, NFFT=NFFT, AZ=AZ, EL=EL)

Segmented generator orders: 70 to 88
sampledWave - Sampled Wave Generator: [##################################################] 100.0%

iSpatFT - Inverse spatial Transform: [##################################################] 100.0%



## Spatial Fourier Transform

In [5]:
Pnm = process.spatFT(Nsft, fftData, quadrature_grid)

## Generate modal radial filters (MF)

In [6]:
dn, _ = gen.radFilter(Nrf, kr, ac, amp_maxdB=limit)

## Plane wave decomposition (PDC) for supplied look directions

In [7]:
Y = process.PWDecomp(Npdc, OmegaL, Pnm, dn)

PWDecomp - Plane Wave Decomposition


## Reconstruct time domain signal (TDT)

In [8]:
impulseResponses = process.iFFT(Y)

iFFT - inverse Fourier Transform


### Make IR causal (flip first & second half)

In [9]:
impulseResponses = np.hstack(np.array_split(impulseResponses, 2, axis=1)[::-1])
# impulseResponses /= 19

## Calculate frequency response

In [10]:
spectrum = 20 * np.log10(np.abs(np.fft.rfft(impulseResponses)))

## Plot time signal and frequency response

In [11]:
plot.plot2D(impulseResponses, type='time', fs=FS)

In [12]:
plot.plot2D(spectrum, type='logFFT', fs=FS)